<a href="https://colab.research.google.com/github/ayush3899/Neural_Network-/blob/master/text_generation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from __future__ import print_function
from keras.callbacks import LambdaCallback
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.optimizers import RMSprop
from keras.utils.data_utils import get_file
import numpy as np
import random
import sys
import io

Using TensorFlow backend.


In [2]:

path = get_file(
    'nietzsche.txt',
    origin='https://s3.amazonaws.com/text-datasets/nietzsche.txt')
with io.open(path, encoding='utf-8') as f:
    text = f.read().lower()
print('corpus length:', len(text))

606208/600901 [==============================] - 0s 0us/step
corpus length: 600893


In [3]:
chars = sorted(list(set(text)))
print('total chars:', len(chars))
char_indices = dict((c, i) for i, c in enumerate(chars))
indices_char = dict((i, c) for i, c in enumerate(chars))

total chars: 57


In [4]:


# cut the text in semi-redundant sequences of maxlen characters
maxlen = 40
step = 3
sentences = []
next_chars = []
for i in range(0, len(text) - maxlen, step):
    sentences.append(text[i: i + maxlen])
    next_chars.append(text[i + maxlen])
print('nb sequences:', len(sentences))

nb sequences: 200285


In [5]:

print('Vectorization...')
x = np.zeros((len(sentences), maxlen, len(chars)), dtype=np.bool)
y = np.zeros((len(sentences), len(chars)), dtype=np.bool)
for i, sentence in enumerate(sentences):
    for t, char in enumerate(sentence):
        x[i, t, char_indices[char]] = 1
    y[i, char_indices[next_chars[i]]] = 1

Vectorization...


In [6]:
# build the model: a single LSTM
print('Build model...')
model = Sequential()
model.add(LSTM(128, input_shape=(maxlen, len(chars))))
model.add(Dense(len(chars), activation='softmax'))

optimizer = RMSprop(lr=0.01)
model.compile(loss='categorical_crossentropy', optimizer=optimizer)

Build model...


In [0]:


def sample(preds, temperature=1.0):
    # helper function to sample an index from a probability array
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)

In [0]:

def on_epoch_end(epoch, _):
    # Function invoked at end of each epoch. Prints generated text.
    print()
    print('----- Generating text after Epoch: %d' % epoch)

    start_index = random.randint(0, len(text) - maxlen - 1)
    for diversity in [0.2, 0.5, 1.0, 1.2]:
        print('----- diversity:', diversity)

        generated = ''
        sentence = text[start_index: start_index + maxlen]
        generated += sentence
        print('----- Generating with seed: "' + sentence + '"')
        sys.stdout.write(generated)

        for i in range(400):
            x_pred = np.zeros((1, maxlen, len(chars)))
            for t, char in enumerate(sentence):
                x_pred[0, t, char_indices[char]] = 1.

            preds = model.predict(x_pred, verbose=0)[0]
            next_index = sample(preds, diversity)
            next_char = indices_char[next_index]

            generated += next_char
            sentence = sentence[1:] + next_char

            sys.stdout.write(next_char)
            sys.stdout.flush()
        print()

print_callback = LambdaCallback(on_epoch_end=on_epoch_end)

model.fit(x, y,
          batch_size=128,
          epochs=60,
          callbacks=[print_callback])

Epoch 1/60
200285/200285 [==============================] - 182s 906us/step - loss: 3.0592

----- Generating text after Epoch: 0
----- diversity: 0.2
----- Generating with seed: " free spirit knows henceforward which
"y"
 free spirit knows henceforward which
"y ho hho i hhi he hhhi i hhi hho i in hi i o hhho i o o i ho i hhhhhi i hhi i hhho hho hhi i hhhhhho i i hho hi hhi i o hi i hho i hhhi i i i i i i i hi i hhhi hhi i hhi i hhhi i o i hhhhhi i hi i hi i i o hhhi o ho i i hi hhi hhi hi hhhi hi hhhhhhhi i hhi hi i o hin i hhhhi hi i  hhhi i hhhi i i i i on hhhhhhhhhi i hho i i hhi hhhhho hi i o hhhhi o i i hi hhhi i i i o hhi i i hhi hi i i i hha i i 
----- diversity: 0.5
----- Generating with seed: " free spirit knows henceforward which
"y"
 free spirit knows henceforward which
"y i  hovin o honthhin hhi o osio eivte h.te esi irhi en ebo o hhh e  hi i hhi ho hi i i hho hhisi i etiritha hititi e o i hhhhhin i hi oini e hitinnshithhiisl i o hho he i i  hhhi hen h o ilhi   isbhhi lon h

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:6: RuntimeWarning: divide by zero encountered in log
  


 innnnannn innn innnninnnninnn innnnonn innn onn onnnonnnn annnnonnn innn on annnnonnn innn inn ann annn onnn onnn innn onnn onnn annninn  innn onnninnnninnn innn annn onn onnnn onn inn onnn onnn onnnninnn h innn innn innn  onnnonnn onnn inn onnn onnn onnn innn inn onnnrinnn innn onnn innn innnnonnn innnnonnn onnnn onnn inn onnn ronnnonnn inn onnnonnnonnn onnn innnnonn 
----- diversity: 0.5
----- Generating with seed: "les must not
be estimated by our standar"
les must not
be estimated by our standarponnn innnrhinnni on ennra  htannnni o onnsonnrhti trin i onnntanlen  tonnon e rhn  an onnntin   an in in an annsen sinn,nnnnha o,nntan hsonnnnennnntatenn intannnnodn annn innsannnnr,n onn,snannnon onnnnan honnninn  ann hn a trann ,nn nonn annnrenn rsenn honnonn annnron onr innrin hinr ,nrinnnnnuann to synn  enn ,n ot on i t innnronnndlirore se  annstonn   o inr innninn onnnnentr e innronnnronn on
----- diversity: 1.0
----- Generating with seed: "les must not
be estimated by our standar"
le